In [1]:
# To retrieve the gff & seq file in NCBI from the GTDB database for the family Methylococcaceae
import os
import matplotlib.pyplot
import pandas as pd
import numpy as np
import subprocess
from Bio import GenBank, SeqIO, AlignIO
import json
import re
import time
import ete3 # 3.1.2
import matplotlib.pyplot as plt
os.chdir("$PATHSorting/S2EBPR.CAS/ComparativeGenome")

In [1]:
import sys
sys.path.append("$PATH/RamanomeSpec/scripts")
import Raman_tree

# Download Methylo family

In [32]:
# read the downloaded GTDB accession
GTDB = pd.read_csv("GTDB_Methy.tsv",sep='\t')
# the reference
# NCBI datasets: https://www.ncbi.nlm.nih.gov/genome/doc/ftpfaq/
GTDB_NCBI = pd.read_csv("assembly_summary_genbank.txt", #assembly_summary_refseq.txt
                        sep='\t',skiprows=1)

In [145]:
def download(GTDB,GTDB_NCBI,pout,download=False):
    """
    GTDB :: the gtdb code of the strains we need
    GTDB_NCBI :: the NCBI datasets for bacteria to be downloaded
    pout :: the output path of json file, [GTDB code] = (NCBI org name, GTDB taxonomy - strain name)
    """
    # download fna and faa files
    dict_name = {} # accession number = key, NCBI strain name = value
    accession = GTDB_NCBI['gbrs_paired_asm'].astype(str)
    assemblyid = GTDB_NCBI.iloc[:,0].astype(str)
    ftp = GTDB_NCBI['ftp_path']
    
    for i in range(GTDB.shape[0]):
        entry = GTDB.iloc[i,0] # the entry from GTDB to NCBI
        ftpi = ftp.loc[np.logical_or(entry == accession,entry==assemblyid)]
        try:
            ftpi = ftpi.tolist()[0] # Series to str
            name = ftpi.split('/')[-1] # gbtk accession number
            name2 = name[3:].split(".")[0] # part of gbtk code to avoid inmatch of GCF/GCA and version .1.2.3 
            dict_name[name2] = (GTDB["ncbi_organism_name"][i],\
                                GTDB["gtdb_taxonomy"][i].split("; s__")[-1],\
                               GTDB["accession"][i]) # align file name and strain name
            #dict_name[name] = (GTDB["ncbi_organism_name"][i],GTDB["gtdb_taxonomy"][i].split("; s__")[-1]) # align file name and strain name
            if download==True:
                for s in ['_cds_from_genomic.fna.gz','_protein.faa.gz']:
                    subprocess.run("curl --remote-name --remote-time {}/{}{}".format(ftpi,name,s),shell=True)
        except IndexError:
            print("{} is not found in reference db".format(entry))
    with open(pout,'w') as f:
        json.dump(dict_name,f)

    # gunzip the downloaded faa and fna
    subprocess.run("gunzip -f *.gz",shell=True)
    print("Successfully downloaded")
    
download(GTDB,GTDB_NCBI,"file_strain.json")

GCF_009828925.1 is not found in reference db
Successfully downloaded


## annotate bin0

In [60]:
def annotate(pin,pout):
    """
    Annotation for bin_0.fna file via KO; the bin_0 is not cds_from_genomics, so require this tep
    pin :: input path for fna file
    pout :: output path for annotated fna file
    """
    # 
    # subprocess.run(["curl","https://rest.kegg.jp/list/ko",">",\
    #                 "/Users/zijianleowang/Desktop/Projects_in_Cornell/RACE/comparative_genomics/ko2ec.txt"])
    C2ko = pd.read_csv("./koall/bin_0/bin_0/ko_mapping.tsv",sep='\t',header=None) # from C_000000 to K02667
    Ko2EC = pd.read_csv("./ko2ec.txt",sep='\t',header=None) # from C_000000 to K02667
    Ko2EC[0] = Ko2EC[0].str.replace("ko:","")
    f = open(pout,'w') # output file for annotated genes
    for seq in SeqIO.parse(pin,format='fasta'):
        try:
            ko = C2ko[C2ko[0]==seq.description][1].values[0]
            description = Ko2EC[Ko2EC[0]==ko][1].values[0]
            seq.description = description
            SeqIO.write(seq,f,'fasta')
        except IndexError:
            "C_00000 not found in bin_0"
    print("Successfully Annotated")
    
annotate("bin_0.fna","./bin_0.annotated.fna")

Successfully Annotated


In [188]:
def get_kseq(pin,MAGsdir):
    """
    get the sequence of customized keywords
    pin :: input path of dict [GTDB code] = NCBI strain name
    MAGsdir: the dir saving all MAGs

    output data: list of tuples, tuple ([genus species strain name, genus species name, gtdb accession number], relevant sequences)

    """
    # get data : (strain name, seq object lists with polyphosphate like ppk, ppx)
    with open(pin,'r') as f:
        dict_name = json.loads( f.readlines()[0] )

    data = []
    for strain in os.scandir(MAGsdir): 
        if (not strain.name.startswith(".DS") and strain.name.endswith("cds_from_genomic.fna")) or "bin_0.annotated.fna" in strain.name:
            seqs = [] # save genes about polyphosphate
            for seq in SeqIO.parse(strain.path,format='fasta'):

                # save the polyphosphate and p transport relevant genes
                if "permease" in seq.description and "phosphate" in seq.description: # pstA/C
                    seqs.append(seq)
                if "signaling" in seq.description and "phosphate" in seq.description: # phoU in NCBI
                    seqs.append(seq)
                if "phoU" in seq.description and "phosphate" in seq.description: # phoU in bin_0
                    seqs.append(seq)
                if "inorganic" in seq.description and "phosphate" in seq.description: # pit
                    seqs.append(seq)
                if "substrate-binding" in seq.description and "phosphate" in seq.description: # pstS
                    seqs.append(seq)
                if "ATP-binding" in seq.description and "phosphate" in seq.description and "ugpC" not in seq.description: # pstB
                    seqs.append(seq)
                if 'polyphos' in seq.description: # ppk1, ppk2, ppx, pap, ppgk
                    seqs.append(seq)

                # save the methane relevant genes
                # if 'methane monooxygenase' in seq.description: # pmo-amo or mmo
                #     seqs.append(seq)
                # if "methanol" in seq.description:    
                # #if 'methanol dehydrogenase' in seq.description or "alcohol oxidase" in seq.description:
                #     seqs.append(seq)
                #     # print(seq.description)
                
            # assign name
            prefix = strain.name.split("_cds_from_genomic.fna")[0] # GTDB code from fna dir
            if "bin_0" not in prefix:
                prefix = prefix[3:].split(".")[0] # part of gbtk code to avoid inmatch of GCF/GCA and version .1.2.3 
            try:
                name = dict_name[prefix]
            except KeyError:
                if "bin_0" in prefix:
                    name = ["bin_0",'bin_0',"bin_0"]
            data.append(( name, seqs))
    # save the GCF - Name
    f = open("dict_name.json",'w')
    json.dump(dict_name,fp=f)
    f.close()
    return data

data = get_kseq("./file_strain.json","./MAGs/")


In [191]:
def get_df(data):
    """
    extract the sequences of strains
    Polyphosphate-AMP phosphotransferase (PAP) and polyphosphate kinase (PPK) 
    were used for designing a novel ATP regeneration system, named the PAP-PPK ATP 
    regeneration system. PAP is an enzyme that catalyzes the phospho-conversion of AMP to ADP, 
    and PPK catalyzes ATP formation from ADP.
    """
    df = pd.DataFrame(np.zeros((len(data),23)),\
                      columns=['strain_name','gtdb_name','gtdb_accession','ppk1','ppk2','ppx','pap','pit','phoU','pstS','pstA_C','pstB','ppgk',
                               'seqppk1','seqppk2','seqppx','seqpap','seqpit','seqphoU','seqpstS','seqpstA_C','seqpstB','seqppgk'\
                              ],\
                      dtype=object)
    for i in range(len(data)): # i-th strain
        sn,gn,gtdb_accession = data[i][0]# strain name, gtdb_name, gtdb_acession
        
        df["strain_name"][i] = sn
        df["gtdb_name"][i] = gn
        df["gtdb_accession"][i] = gtdb_accession
        for seq in data[i][1]: # get gene information
            des = seq.description # the description string
            dna = seq.seq
            try:
                protein = re.findall(r'\[protein=(.*?)\]',des)[0]
            except IndexError: # bin_0 starts with c_0000 and has different format compare to NCBI data
                protein = des

            try:
                gene = re.findall(r'\[gene=(.*?)\]',des)[0]
            except IndexError: # no gene field found
                if "ppk1" in protein or "polyphosphate kinase" in protein:
                    gene = "ppk1"
                elif "ppx" in protein or "exopolyphosphatase" in protein:
                    gene = "ppx"
                elif 'kinase 2' in protein or "ppk2" in protein:
                    gene = 'ppk2'
                elif "inorganic" in protein:
                    gene = 'pit'
                elif "signaling" in protein or "phoU" in protein:
                    gene = 'phoU'
                elif "substrate" in protein:
                    gene = 'pstS'
                elif "permease" in protein:
                    gene = 'pstA_C'
                elif "ATP" in protein:
                    gene = 'pstB'
                elif "glucokinase" in protein:
                    gene = 'ppgk'

            if gene =='pstA' or gene == 'pstC':
                gene = 'pstA_C'
            
            try:
                df[gene][i] = 1
                df['seq{}'.format(gene)][i] = dna
            except KeyError: # 
                print('wrong key')
    # remove duplicates
    df = df.drop_duplicates("gtdb_name")
    return df

df = get_df(data)

## Save the processed genetic information: whether there is ppk1, ppk2, etc. and their sequences

In [192]:
df.to_csv("genes.seqs.csv")

In [134]:
# save df to fasta
def save(df,genelist):
    for gene in genelist:
        f = open("%s.fasta"%gene,'w')
        for i in range(df.shape[0]): # i denotes the strain
            YOR = df[gene].iloc[i] # gene exists or not
            if YOR == 1:
                seq = str(df['seq%s'%gene].iloc[i])
                name = df['strain_name'].iloc[i].split(" ")[:3]
                name = "_".join(name)
                print(">%s\n%s\n"%(name,seq),file=f)
    f.close()
    print("Successfully saved the fasta files of various genes in the selected strain list")
genelist = ["ppk1","ppk2","ppx","pap",'pit','phoU','pstS','pstA_C','pstB']
save(df,genelist)

Successfully saved the fasta files of various genes in the selected strain list


# Create all the phosphorus-relevant genes tree xml

In [35]:
Pname = 'allPgenes'
# combine all of the genes together 
df = pd.read_csv("genes.seqs.csv",index_col=0)
col = df.columns.tolist()
col = [i for i in col if 'seq' in i]
df2 = df[col].apply("".join,axis=1)
df2 = df2.str.replace("0.0","")
df2.name = Pname
df3 = df.copy().drop(labels=col,axis=1)
dfall = pd.concat([df3,df2],axis=1)

# save the data
f = open("%s.fasta"%Pname,'w')
for i in range(dfall.shape[0]):
    seq = str(dfall[Pname].iloc[i])
    name = dfall["strain_name"].iloc[i].split(" ")[:3]
    name = "_".join(name)
    print(">%s\n%s\n"%(name,seq),file=f)
f.close()

# align the seq
subprocess.run("mafft %s.fasta > %s.aligned.fasta"%(Pname,Pname),shell=True)
# create tree
subprocess.run(["/Users/zijianleowang/raxml-ng_v1.1.0_macos_x86_64/raxml-ng", "--seed", "12338", "--model", "GTR", 
                        "--msa", "%s.aligned.fasta"%Pname, "--prefix","%s"%Pname,"--redo"])

CompletedProcess(args=['/Users/zijianleowang/raxml-ng_v1.1.0_macos_x86_64/raxml-ng', '--seed', '12338', '--model', 'GTR', '--msa', 'allPgenes.aligned.fasta', '--prefix', 'allPgenes', '--redo'], returncode=0)

# Visualize each of the gene trees

In [135]:
def create_tree(genelist,):
    """
    align the sequence and create the tree
    """
    # Align the sequences
    # using MAFFT
    # https://mafft.cbrc.jp/alignment/software/macstandard.html
    for gene in genelist:
        subprocess.run("mafft %s.fasta > %s.aligned.fasta"%(gene,gene),\
                       shell=True) #["mafft","%s.fasta"%gene,">","%s.aligned.fasta"%gene]
        #time.sleep(2)
    # RAXML-ng to compute distance
    # https://github.com/amkozlov/raxml-ng/releases
    # https://formulae.brew.sh/formula/raxml-ng
    success = []
    fail = []
    for gene in genelist:
        i = 0
        with open("%s.aligned.fasta"%gene,'r') as f:
            for line in f:
                if line.startswith(">"):
                    i += 1
        if i >4: # create tree if it has at least 4 strains
            success.append(gene)
            subprocess.run(["/Users/zijianleowang/raxml-ng_v1.1.0_macos_x86_64/raxml-ng", "--seed", "12338", "--model", "GTR", 
                        "--msa", "%s.aligned.fasta"%gene, "--prefix","%s"%gene,"--redo"])
        else:
            fail.append(gene)
    print("Successfully created the tree ")
    return success, fail
genelist = ["ppk1","ppk2","ppx","pap",'pit','phoU','pstS','pstA_C','pstB']
success, fail = create_tree(genelist)

Successfully created the tree 


In [88]:
def viz_tree(success,df,pathin,pathout):
    """
    visualize the trees 
    """
    # generate the genus name
    taxname = []
    for i in range(df.shape[0]):
        name = df['strain_name'].iloc[i].split(" ")[:2]
        name = "_".join(name)
        taxname.append(name)
    genus = [i.split('_')[0] for i in taxname]
    genus = sorted(set(genus))
    print(len(genus),"genus")
    # generate color range
    color = ["#D1BBD7","#AE76A3","#BB2E72","#196580","#5289C7","#7BAFDE","#4EB265","#90C987",\
            "#CAE0AB","#F7F056","#F6C141","#F1932D","#E8601C","#DC050C","#777777","#666666","#03fcec"]
    dict_gc = {}
    for i, g in enumerate(genus):
        dict_gc[g] = color[i] # dictionary of genus to color
    with open("dict_gc.json",'w') as f:
        json.dump(dict_gc,fp=f)
    # ETE3 to visualize tree
    trees = []
    for gene in success:
        in_path=os.path.join(pathin,"%s.raxml.bestTree"%gene)
        print(in_path)
        tree = Raman_tree.read_tree(in_path,
                                    sep="_",
                                    special={"bin_0":"M. multiphosphori"},
                                    title=gene)
        tree.set_color_range(dict_gc)
        tree.tree_g.render(os.path.join(pathout,"%s_tree.png"%gene),
                    tree_style=tree.ts,w=3000,dpi=600)

        trees.append(tree)
    # save the label legend
    # plot dict_gc

    fig, ax = plt.subplots(1,1,figsize=(2,10))
    plt.axis('off') 
    x1, x2 = 0, 1
    k = list(dict_gc.keys())#keys

    ax.set_xlim((-.5,1.5))
    ax.set_ylim((-1,19))

    for i in np.arange(len(k)):
        ax.fill_between((x1,x2),i,i+1,color=dict_gc[k[i]])
        ax.text(x1-.5,i+.5,k[i],ha='right',fontweight='bold',fontsize='xx-large',fontstyle='italic')
    fig.savefig(os.path.join(pathout,'Label.png'),
                bbox_inches='tight',quality=300)
    plt.close()

    return trees

In [153]:
imp.reload(Raman_tree)
df = pd.read_csv("genes.seqs.csv",index_col=0)
genelists = ["ppk1","ppk2","ppx","pit","phoU","pstA_C","pstB","pstS","pap","allPgenes"]
trees = viz_tree(genelists,df,
         pathin="./Tree/Newick_FASTA/",
         pathout="Tree/Image/")

17 genus
./Tree/Newick_FASTA/ppk1.raxml.bestTree
./Tree/Newick_FASTA/ppk2.raxml.bestTree
./Tree/Newick_FASTA/ppx.raxml.bestTree
./Tree/Newick_FASTA/pit.raxml.bestTree
./Tree/Newick_FASTA/phoU.raxml.bestTree
./Tree/Newick_FASTA/pstA_C.raxml.bestTree
./Tree/Newick_FASTA/pstB.raxml.bestTree
./Tree/Newick_FASTA/pstS.raxml.bestTree
./Tree/Newick_FASTA/pap.raxml.bestTree
./Tree/Newick_FASTA/allPgenes.raxml.bestTree
